In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

cap = cv2.VideoCapture('vid1_IR.avi')
while(cap.isOpened()):
    ret, frame = cap.read()
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('IR', G)
    if cv2.waitKey(1) & 0xFF == ord ('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [12]:
import cv2
import numpy as np

cap = cv2.VideoCapture('vid1_IR.avi')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # binaryzacja
    _, binary = cv2.threshold(G, 35, 255, cv2.THRESH_BINARY)

    # filtracja
    binary = cv2.medianBlur(binary, 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # indeksacja
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)

    output = frame.copy()
    boxes = []

    # parametry ograniczające
    min_area = 1500
    min_aspect_ratio = 1.5
    max_width = 150  # ograniczenie szerokości sylwetki żeby prostokąty obok sie nie łączyły

    for i in range(1, num_labels):  # pomijamy tło
        x, y, w, h, area = stats[i]

        if area < min_area:
            continue

        aspect_ratio = h / float(w)
        if aspect_ratio < min_aspect_ratio or w > max_width:
            continue

        boxes.append([x, y, x + w, y + h])  # [x1, y1, x2, y2]
    # sortowanie po gorze prostokatow
    boxes.sort(key=lambda b: b[1])

    merged = []

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        merged_box = [x1, y1, x2, y2]  # aktualny prostokąt do potencjalnego rozszerzenia

        for j in range(i + 1, len(boxes)):
            x1b, y1b, x2b, y2b = boxes[j]

            vertical_gap = y1b - merged_box[3]  # różnica pionowa między kolejnym prostokątem a dolną krawędzią bieżącego

            # czy prostokąty są blisko w pionie
            if 0 <= vertical_gap <= 20:
                # overlap między prostokątami
                overlap_w = min(merged_box[2], x2b) - max(merged_box[0], x1b)

                if overlap_w > 0:
                    min_width = min(merged_box[2] - merged_box[0], x2b - x1b)
                    # stosunek overlapu do minimalnej szerokości
                    overlap_ratio = overlap_w / float(min_width)

                    # overlap wystarczająco duży
                    if overlap_ratio >= 0.6:
                        # rozszerzamy prostokat
                        merged_box[0] = min(merged_box[0], x1b)  # lewy 
                        merged_box[1] = min(merged_box[1], y1b)  # gorny
                        merged_box[2] = max(merged_box[2], x2b)  # prawy
                        merged_box[3] = max(merged_box[3], y2b)  # dolny

        merged.append(merged_box)

    for box in merged:
        x1, y1, x2, y2 = box
        cv2.rectangle(output, (x1, y1), (x2, y2), (255, 0, 0), 2)

    cv2.imshow('IR', output)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1